In [1]:
!git clone --recursive https://github.com/junyi42/monst3r /root/jupyter/monst3r


fatal: destination path '/root/jupyter/monst3r' already exists and is not an empty directory.


In [2]:
import torch

In [3]:
# DUST3R relies on RoPE positional embeddings for which you can compile some cuda kernels for faster runtime.
%cd /root/jupyter/monst3r/croco/models/curope/
!python setup.py build_ext --inplace

/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/monst3r/croco/models/curope
running build_ext
W1014 17:24:08.508000 30 site-packages/torch/utils/cpp_extension.py:615] Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
W1014 17:24:09.083000 30 site-packages/torch/utils/cpp_extension.py:507] The detected CUDA version (12.1) has a minor version mismatch with the version that was used to compile PyTorch (12.8). Most likely this shouldn't be a problem.
W1014 17:24:09.084000 30 site-packages/torch/utils/cpp_extension.py:517] There are no clang++ version bounds defined for CUDA version 12.1
building 'curope' extension
creating build
creating build/temp.linux-x86_64-cpython-310
clang -pthread -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -I/tools/deps/include -I/tools/deps/include/ncursesw -I/tools/deps/libedit/include -g0 -fPIC -I/usr/local/lib/python3.10/site-packages/torch

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [5]:
def get_3D_model_from_scene(outdir, silent, scene, min_conf_thr=3, as_pointcloud=False, mask_sky=False,
                            clean_depth=False, transparent_cams=False, cam_size=0.05, show_cam=True, save_name=None, thr_for_init_conf=True):
    """
    extract 3D_model (glb file) from a reconstructed scene
    """
    if scene is None:
        return None
    # post processes
    if clean_depth:
        scene = scene.clean_pointcloud()
    if mask_sky:
        scene = scene.mask_sky()

    # get optimized values from scene
    rgbimg = scene.imgs
    focals = scene.get_focals().cpu()
    cams2world = scene.get_im_poses().cpu()
    # 3D pointcloud from depthmap, poses and intrinsics
    pts3d = to_numpy(scene.get_pts3d(raw_pts=True))
    scene.min_conf_thr = min_conf_thr
    scene.thr_for_init_conf = thr_for_init_conf
    msk = to_numpy(scene.get_masks())
    cmap = pl.get_cmap('viridis')
    cam_color = [cmap(i/len(rgbimg))[:3] for i in range(len(rgbimg))]
    cam_color = [(255*c[0], 255*c[1], 255*c[2]) for c in cam_color]
    return convert_scene_output_to_glb(outdir, rgbimg, pts3d, msk, focals, cams2world, as_pointcloud=as_pointcloud,
                                        transparent_cams=transparent_cams, cam_size=cam_size, show_cam=show_cam, silent=silent, save_name=save_name,
                                        cam_color=cam_color)

In [ ]:

def get_reconstructed_scene(args, outdir, model, device, silent, image_size, schedule, niter, min_conf_thr,
                            as_pointcloud, mask_sky, clean_depth, transparent_cams, cam_size, show_cam, scenegraph_type, winsize, refid, 
                            seq_name, new_model_weights, temporal_smoothing_weight, translation_weight, shared_focal, 
                            flow_loss_weight, flow_loss_start_iter, flow_loss_threshold, use_gt_mask, fps, num_frames):
    """
    from a list of images, run dust3r inference, global aligner.
    then run get_3D_model_from_scene
    """
    translation_weight = float(translation_weight)
    if new_model_weights != args.weights:
        model = AsymmetricCroCo3DStereo.from_pretrained(new_model_weights).to(device)
    model.eval()
    if seq_name != "NULL":
        dynamic_mask_path = f'data/davis/DAVIS/masked_images/480p/{seq_name}'
    else:
        dynamic_mask_path = None

    imgs = torch.load("/root/jupyter/data/preprocessed_data.pth")
        
    if len(imgs) == 1:
        imgs = [imgs[0], copy.deepcopy(imgs[0])]
        imgs[1]['idx'] = 1
    if scenegraph_type == "swin" or scenegraph_type == "swinstride" or scenegraph_type == "swin2stride":
        scenegraph_type = scenegraph_type + "-" + str(winsize) + "-noncyclic"
    elif scenegraph_type == "oneref":
        scenegraph_type = scenegraph_type + "-" + str(refid)

    pairs = make_pairs(imgs, scene_graph=scenegraph_type, prefilter=None, symmetrize=True)
    output = inference(pairs, model, device, batch_size=args.batch_size, verbose=not silent)
    # TODO YYJ del model
    if len(imgs) > 2:
        mode = GlobalAlignerMode.PointCloudOptimizer
        scene = global_aligner(output, device=device, mode=mode, verbose=not silent, shared_focal = shared_focal, temporal_smoothing_weight=temporal_smoothing_weight, translation_weight=translation_weight,
                               flow_loss_weight=flow_loss_weight, flow_loss_start_epoch=flow_loss_start_iter, flow_loss_thre=flow_loss_threshold, use_self_mask=not use_gt_mask,
                               num_total_iter=niter, empty_cache= len(filelist) > 72, batchify=not (args.not_batchify or args.window_wise),
                               window_wise=args.window_wise, window_size=args.window_size, window_overlap_ratio=args.window_overlap_ratio,
                               prev_video_results=prev_video_results)
    else:
        mode = GlobalAlignerMode.PairViewer
        scene = global_aligner(output, device=device, mode=mode, verbose=not silent)
    lr = 0.01

    if mode == GlobalAlignerMode.PointCloudOptimizer:
        if args.window_wise:
            scene.compute_window_wise_alignment(init='mst', niter=niter, schedule=schedule, lr=lr)
        else:
            scene.compute_global_alignment(init='mst', niter=niter, schedule=schedule, lr=lr)

    if args.window_wise and args.prev_output_dir is not None:
        scene.clean_prev_results()
        
    save_folder = f'{args.output_dir}/{seq_name}'  #default is 'demo_tmp/NULL'
    os.makedirs(save_folder, exist_ok=True)
    outfile = get_3D_model_from_scene(save_folder, silent, scene, min_conf_thr, as_pointcloud, mask_sky,
                            clean_depth, transparent_cams, cam_size, show_cam)

    poses = scene.save_tum_poses(f'{save_folder}/pred_traj.txt')
    K = scene.save_intrinsics(f'{save_folder}/pred_intrinsics.txt')
    depth_maps = scene.save_depth_maps(save_folder)
    dynamic_masks = scene.save_dynamic_masks(save_folder)
    conf = scene.save_conf_maps(save_folder)
    init_conf = scene.save_init_conf_maps(save_folder)
    rgbs = scene.save_rgb_imgs(save_folder)
    enlarge_seg_masks(save_folder, kernel_size=5 if use_gt_mask else 3) 

    # also return rgb, depth and confidence imgs
    # depth is normalized with the max value for all images
    # we apply the jet colormap on the confidence maps
    rgbimg = scene.imgs
    depths = to_numpy(scene.get_depthmaps())
    confs = to_numpy([c for c in scene.im_conf])
    init_confs = to_numpy([c for c in scene.init_conf_maps])
    cmap = pl.get_cmap('jet')
    depths_max = max([d.max() for d in depths])
    depths = [cmap(d/depths_max) for d in depths]
    confs_max = max([d.max() for d in confs])
    confs = [cmap(d/confs_max) for d in confs]
    init_confs_max = max([d.max() for d in init_confs])
    init_confs = [cmap(d/init_confs_max) for d in init_confs]

    imgs = []
    for i in range(len(rgbimg)):
        imgs.append(rgbimg[i])
        imgs.append(rgb(depths[i]))
        imgs.append(rgb(confs[i]))
        imgs.append(rgb(init_confs[i]))

    # if two images, and the shape is same, we can compute the dynamic mask
    if len(rgbimg) == 2 and rgbimg[0].shape == rgbimg[1].shape:
        motion_mask_thre = 0.35
        error_map = get_dynamic_mask_from_pairviewer(scene, both_directions=True, output_dir=args.output_dir, motion_mask_thre=motion_mask_thre)
        # imgs.append(rgb(error_map))
        # apply threshold on the error map
        normalized_error_map = (error_map - error_map.min()) / (error_map.max() - error_map.min())
        error_map_max = normalized_error_map.max()
        error_map = cmap(normalized_error_map/error_map_max)
        imgs.append(rgb(error_map))
        binary_error_map = (normalized_error_map > motion_mask_thre).astype(np.uint8)
        imgs.append(rgb(binary_error_map*255))

    return scene, outfile, imgs

In [23]:
!sed -i "s/torch.load(model_path, map_location='cpu')/torch.load(model_path, map_location='cpu', weights_only=False)/" /root/jupyter/monst3r/dust3r/model.py

In [4]:
import argparse
import torch
torch.serialization.add_safe_globals([argparse.Namespace])

In [5]:
%ls

__pycache__/  host_nb.py  jupyter@


In [6]:
%ls /root/jupyter/monst3r

LICENSE       datasets_preprocess/  requirements.txt
README.md     demo.py               requirements_optional.txt
assets/       demo_data/            third_party/
checkpoints/  depth_metric.ipynb    viser/
croco/        dust3r/
data/         launch.py


In [3]:
%cd /root/jupyter/monst3r
INPUT_DIR = "/root/jupyter/data/resized_classified/iphone6/landscape"
OUTPUT_DIR = "/root/jupyter/data/demo_tmp"
!python demo.py --input {INPUT_DIR} --output_dir {OUTPUT_DIR} --seq_name iphone6

/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/monst3r
Initialized new /root/.evo/settings.json
Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead
/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/monst3r/dust3r/cloud_opt/base_opt.py:399: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/monst3r/dust3r/cloud_opt/base_opt.py:416: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
... loading model from checkpoints/MonST3R_PO-TA-S-W_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), enc_embed_dim=102

In [ ]:
# --------------------------------------------------------
# Imports and setup
# --------------------------------------------------------
%cd /root/jupyter/monst3r
import sys
# Add monst3r directory to Python path to find the modules
if 'monst3r' not in sys.path:
    sys.path.append('monst3r')

import argparse
import math
import os
import torch
import numpy as np
import tempfile
import functools
import copy

from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.image_pairs import make_pairs
from dust3r.utils.image import load_images, load_prev_video_results, rgb, enlarge_seg_masks
from dust3r.utils.device import to_numpy
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from dust3r.utils.viz_demo import convert_scene_output_to_glb, get_dynamic_mask_from_pairviewer
import matplotlib.pyplot as pl
import cv2

pl.ion()
torch.backends.cuda.matmul.allow_tf32 = True  # for gpu >= Ampere and pytorch >= 1.12


# --------------------------------------------------------
# Function definitions from the Gradio demo script
# --------------------------------------------------------

def get_3D_model_from_scene(outdir, silent, scene, min_conf_thr=3, as_pointcloud=False, mask_sky=False,
                            clean_depth=False, transparent_cams=False, cam_size=0.05, show_cam=True, save_name=None, thr_for_init_conf=True):
    """
    extract 3D_model (glb file) from a reconstructed scene
    """
    if scene is None:
        return None
    # post processes
    if clean_depth:
        scene = scene.clean_pointcloud()
    if mask_sky:
        scene = scene.mask_sky()

    # get optimized values from scene
    rgbimg = scene.imgs
    focals = scene.get_focals().cpu()
    cams2world = scene.get_im_poses().cpu()
    # 3D pointcloud from depthmap, poses and intrinsics
    pts3d = to_numpy(scene.get_pts3d(raw_pts=True))
    scene.min_conf_thr = min_conf_thr
    scene.thr_for_init_conf = thr_for_init_conf
    msk = to_numpy(scene.get_masks())
    cmap = pl.get_cmap('viridis')
    cam_color = [cmap(i/len(rgbimg))[:3] for i in range(len(rgbimg))]
    cam_color = [(255*c[0], 255*c[1], 255*c[2]) for c in cam_color]
    return convert_scene_output_to_glb(outdir, rgbimg, pts3d, msk, focals, cams2world, as_pointcloud=as_pointcloud,
                                        transparent_cams=transparent_cams, cam_size=cam_size, show_cam=show_cam, silent=silent, save_name=save_name,
                                        cam_color=cam_color)

def get_reconstructed_scene(args, outdir, model, device, silent, image_size, schedule, niter, min_conf_thr,
                            as_pointcloud, mask_sky, clean_depth, transparent_cams, cam_size, show_cam, scenegraph_type, winsize, refid, 
                            seq_name, new_model_weights, temporal_smoothing_weight, translation_weight, shared_focal, 
                            flow_loss_weight, flow_loss_start_iter, flow_loss_threshold, use_gt_mask, fps, num_frames):
    """
    from a list of images, run dust3r inference, global aligner.
    then run get_3D_model_from_scene
    """
    translation_weight = float(translation_weight)
    if new_model_weights != args.weights:
        model = AsymmetricCroCo3DStereo.from_pretrained(new_model_weights).to(device)
    model.eval()

    # <<< YOUR MODIFICATION IS HERE
    # Using your specified method to load preprocessed data
    print("Loading preprocessed data from '/root/jupyter/data/preprocessed_data.pth'")
    imgs = torch.load("/root/jupyter/data/preprocessed_data.pth", weights_only=False)
    # >>> END OF MODIFICATION
        
    if len(imgs) == 1:
        imgs = [imgs[0], copy.deepcopy(imgs[0])]
        imgs[1]['idx'] = 1
    if scenegraph_type == "swin" or scenegraph_type == "swinstride" or scenegraph_type == "swin2stride":
        scenegraph_type = scenegraph_type + "-" + str(winsize) + "-noncyclic"
    elif scenegraph_type == "oneref":
        scenegraph_type = scenegraph_type + "-" + str(refid)

    pairs = make_pairs(imgs, scene_graph=scenegraph_type, prefilter=None, symmetrize=True)
    output = inference(pairs, model, device, batch_size=args.batch_size, verbose=not silent)
    
    # The rest of the function remains the same...
    if len(imgs) > 2:
        mode = GlobalAlignerMode.PointCloudOptimizer
        scene = global_aligner(output, device=device, mode=mode, verbose=not silent, shared_focal = shared_focal, temporal_smoothing_weight=temporal_smoothing_weight, translation_weight=translation_weight,
                               flow_loss_weight=flow_loss_weight, flow_loss_start_epoch=flow_loss_start_iter, flow_loss_thre=flow_loss_threshold, use_self_mask=not use_gt_mask,
                               num_total_iter=niter, empty_cache= len(imgs) > 72, batchify=not (args.not_batchify or args.window_wise),
                               window_wise=args.window_wise, window_size=args.window_size, window_overlap_ratio=args.window_overlap_ratio,
                               prev_video_results=None) # prev_video_results is not used with preprocessed data
    else:
        mode = GlobalAlignerMode.PairViewer
        scene = global_aligner(output, device=device, mode=mode, verbose=not silent)
    lr = 0.01

    if mode == GlobalAlignerMode.PointCloudOptimizer:
        if args.window_wise:
            scene.compute_window_wise_alignment(init='mst', niter=niter, schedule=schedule, lr=lr)
        else:
            scene.compute_global_alignment(init='mst', niter=niter, schedule=schedule, lr=lr)

    if args.window_wise and args.prev_output_dir is not None:
        scene.clean_prev_results()
        
    save_folder = f'{args.output_dir}/{seq_name}'
    os.makedirs(save_folder, exist_ok=True)
    outfile = get_3D_model_from_scene(save_folder, silent, scene, min_conf_thr, as_pointcloud, mask_sky,
                            clean_depth, transparent_cams, cam_size, show_cam)

    poses = scene.save_tum_poses(f'{save_folder}/pred_traj.txt')
    K = scene.save_intrinsics(f'{save_folder}/pred_intrinsics.txt')
    depth_maps = scene.save_depth_maps(save_folder)
    dynamic_masks = scene.save_dynamic_masks(save_folder)
    conf = scene.save_conf_maps(save_folder)
    init_conf = scene.save_init_conf_maps(save_folder)
    rgbs = scene.save_rgb_imgs(save_folder)
    enlarge_seg_masks(save_folder, kernel_size=5 if use_gt_mask else 3) 

    rgbimg = scene.imgs
    depths = to_numpy(scene.get_depthmaps())
    confs = to_numpy([c for c in scene.im_conf])
    init_confs = to_numpy([c for c in scene.init_conf_maps])
    
    imgs_for_viz = []

    return scene, outfile, imgs_for_viz

# --------------------------------------------------------
# Main execution logic for the notebook
# --------------------------------------------------------

# 1. Setup arguments (instead of argparse)
class Args:
    def __init__(self):
        # You may need to download the weights first.
        # The default path is from the original script.
        self.weights = 'checkpoints/MonST3R_PO-TA-S-W_ViTLarge_BaseDecoder_512_dpt.pth'
        self.model_name = 'Junyi42/MonST3R_PO-TA-S-W_ViTLarge_BaseDecoder_512_dpt'
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.output_dir = '/root/jupyter/output'
        self.prev_output_dir = None
        self.prev_output_index = None
        self.silent = False
        # # --- IMPORTANT: Change this to your image folder or video file ---
        # self.input_dir = '/root/jupy' 
        self.seq_name = 'huaca' # This will be the output subfolder name
        self.use_gt_davis_masks = False
        self.not_batchify = False
        self.real_time = False
        self.window_wise = False
        self.window_size = 100
        self.window_overlap_ratio = 0.5
        self.batch_size = 100 # Lower this if you get CUDA out-of-memory errors
        self.fps = 0
        self.num_frames = 200
        self.image_size = 512

args = Args()

# Create output directory
tmpdirname = args.output_dir
os.makedirs(tmpdirname, exist_ok=True)
tempfile.tempdir = tmpdirname

print(f'Using device: {args.device}')
print('Outputting results in:', tmpdirname)

# 2. Load model
if args.weights and os.path.exists(args.weights):
    weights_path = args.weights
    print(f"Loading weights from {weights_path}")
else:
    print(f"Weights file not found at '{args.weights}'. Attempting to load from Hugging Face Hub model name '{args.model_name}'.")
    weights_path = args.model_name

model = AsymmetricCroCo3DStereo.from_pretrained(weights_path).to(args.device)
model.eval()


# 4. Run reconstruction
print("Starting reconstruction...")
scene, outfile, imgs = get_reconstructed_scene(
    args=args,
    outdir=tmpdirname,
    model=model,
    device=args.device,
    silent=args.silent,
    image_size=args.image_size,
    # filelist=input_files,
    schedule='linear',
    niter=300,
    min_conf_thr=1.1,
    as_pointcloud=True,
    mask_sky=False,
    clean_depth=True,
    transparent_cams=False,
    cam_size=0.05,
    show_cam=True,
    scenegraph_type='swinstride',
    winsize=5,
    refid=0,
    seq_name=args.seq_name,
    new_model_weights=args.weights,
    temporal_smoothing_weight=0.01,
    translation_weight='1.0',
    shared_focal=True,
    flow_loss_weight=0.01,
    flow_loss_start_iter=0.1,
    flow_loss_threshold=25,
    use_gt_mask=args.use_gt_davis_masks,
    fps=args.fps,
    num_frames=args.num_frames,
)

print("\nProcessing completed.")
print(f"Output maps and data saved in: {os.path.join(tmpdirname, args.seq_name)}")
if outfile:
    print(f"3D model saved to: {outfile}")


/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/monst3r
Using device: cuda
Outputting results in: /root/jupyter/output
Loading weights from checkpoints/MonST3R_PO-TA-S-W_ViTLarge_BaseDecoder_512_dpt.pth
... loading model from checkpoints/MonST3R_PO-TA-S-W_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), enc_embed_dim=1024, enc_depth=24, enc_num_heads=16, dec_embed_dim=768, dec_depth=12, dec_num_heads=12, freeze='encoder', landscape_only=False)
Freezing encoder parameters
<All keys matched successfully>
Starting reconstruction...
Loading preprocessed data from '/root/jupyter/data/preprocessed_data.pth'
>> Inference with model on 27320 image pairs


100%|██████████| 27320/27320 [1:04:35<00:00,  7.05it/s]


In [8]:
%ls /root/jupyter/monst3r/data

download_bonn.sh             download_spring.sh
download_ckpt.sh             download_tartanair.py
download_davis.py            download_tartanair.sh
download_dynamic_replica.sh  download_training_zipfiles.txt
download_kitti.sh            download_tum_dynamics.sh
download_nyuv2.sh            download_waymo.sh
download_pointodyssey.sh     evaluation_script.md
download_scannetv2.sh        prepare_training.md
download_sintel.sh


In [9]:
%cd /root/jupyter/monst3r/data
!bash download_ckpt.sh

/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/monst3r/data
Downloading...
From (original): https://drive.google.com/uc?id=1Z1jO_JmfZj0z3bgMvCwqfUhyZ1bIbc9E
From (redirected): https://drive.google.com/uc?id=1Z1jO_JmfZj0z3bgMvCwqfUhyZ1bIbc9E&confirm=t&uuid=12fabc9d-a0ec-40a2-adfa-0017fd7efa7c
To: /__modal/volumes/vo-05VlmA5Lclgm1HklUu8tDq/monst3r/checkpoints/MonST3R_PO-TA-S-W_ViTLarge_BaseDecoder_512_dpt.pth
100%|██████████████████████████████████████| 2.29G/2.29G [00:28<00:00, 81.2MB/s]
--2025-10-14 17:33:27--  https://www.dropbox.com/s/4j4z58wuv8o0mfz/models.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.11.18, 2620:100:6050:18::a27d:b12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.11.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/1gtoiul6tr71ag8xjet5v/models.zip?rlkey=lu59i7g742lo6o2yt15f8t96r [following]
--2025-10-14 17:33:28--  https://www.dropbox.com/scl/fi/1gtoiul6tr71ag8xjet5v/models.zip?rlkey=l

In [12]:
%ls /root/jupyter/monst3r/data

download_bonn.sh             download_spring.sh
download_ckpt.sh             download_tartanair.py
download_davis.py            download_tartanair.sh
download_dynamic_replica.sh  download_training_zipfiles.txt
download_kitti.sh            download_tum_dynamics.sh
download_nyuv2.sh            download_waymo.sh
download_pointodyssey.sh     evaluation_script.md
download_scannetv2.sh        prepare_training.md
download_sintel.sh


In [16]:
%ls /root/jupyter/

CUT3R/  data/  monst3r/
